\begin{align}
 S(X) = \frac{\frac{k^{(1/m)}}{k^{(1/n)}} -\frac{\|X\|_m}{\|X\|_n} } {\frac{k^{(1/m)}}{k^{(1/n)}}-1}
\end{align}

- k: co-occurrance vector dimension: number of words in a concept
- m: 1 why? 빠를거 같아서
- n: 2 빠를거 같아서
- X: co-occurrance vector (in all documents)

In [1]:
from config import document_fname, word2concept_fname

## concept to words

In [2]:
with open(word2concept_fname, encoding='utf-8') as f:
    for _ in range(10):
        print(next(f).strip())

sale,28
romney,70
anti-cancer,68
delete,33
concluded,97
societal,16
extinction,47
priority,50
volatile,14
giles,81


In [3]:
from collections import defaultdict

concept_to_words = defaultdict(lambda: [])
with open(word2concept_fname, encoding='utf-8') as f:
    for row in f:
        cols = row.strip().split(',')
        concept = int(cols[-1])
        words = ','.join(cols[:-1])
        concept_to_words[concept].append(words)
        

## co-occurrance matrix

In [7]:
from utils import get_process_memory

In [9]:
print("""
co-occur(1, 2) = 5

[[0, 1, 2, 0, 5], #c=1
 [0, 1, 3, 1, 0], # c= 1 
 ....             # c = 2
]""")


co-occur(1, 2) = 5

[[0, 1, 2, 0, 5], #c=1
 [0, 1, 3, 1, 0], # c= 1 
 ....             # c = 2
]


In [11]:
import sys

In [13]:
from collections import Counter
import sys

M_term_doc = defaultdict(lambda: {})
with open(document_fname, encoding='utf-8') as f:
    for d, doc in enumerate(f):
        tf = Counter(doc.split())
        for t, freq in tf.items():
            M_term_doc[t][d] = freq
        if d % 1000 == 0: 
            sys.stdout.write('\r inserting ... %d docs, mem= %.3f Gb' %(d+1, get_process_memory()))

 inserting ... 203001 docs, mem= 2.566 Gb

In [28]:
def cooccurrence(w1, w2):
    docs1 = M_term_doc.get(w1, {})
    docs2 = M_term_doc.get(w2, {})
    cooccurrence = 0
    for d1, tf_d1w1 in docs1.items():
        tf_d1w2 = docs2.get(d1, 0)
        if not tf_d1w2:
            continue
        cooccurrence += max(tf_d1w1, tf_d1w2)
    return cooccurrence

M_cooccurrence = defaultdict(lambda: {})
for concept, words in concept_to_words.items():
    print('concept= %d (%d words) ... ' % (concept, len(words)), end='')
    for w1 in words:
        for w2 in words:
            if w1 <= w2: continue
            cooc = cooccurrence(w1, w2)
            M_cooccurrence[w1][w2] = cooc
            M_cooccurrence[w2][w1] = cooc
    print('done. mem= %.3f Gb' % get_process_memory())

concept= 0 (505 words) ... done. mem= 2.799 Gb
concept= 1 (387 words) ... done. mem= 2.801 Gb
concept= 2 (385 words) ... done. mem= 2.808 Gb
concept= 3 (449 words) ... done. mem= 2.815 Gb
concept= 4 (520 words) ... done. mem= 2.824 Gb
concept= 5 (327 words) ... done. mem= 2.824 Gb
concept= 6 (187 words) ... done. mem= 2.826 Gb
concept= 7 (457 words) ... done. mem= 2.836 Gb
concept= 8 (512 words) ... done. mem= 2.846 Gb
concept= 9 (644 words) ... done. mem= 2.857 Gb
concept= 10 (209 words) ... done. mem= 2.857 Gb
concept= 11 (184 words) ... done. mem= 2.858 Gb
concept= 12 (303 words) ... done. mem= 2.859 Gb
concept= 13 (658 words) ... done. mem= 2.876 Gb
concept= 14 (533 words) ... done. mem= 2.887 Gb
concept= 15 (334 words) ... done. mem= 2.887 Gb
concept= 16 (515 words) ... done. mem= 2.901 Gb
concept= 17 (261 words) ... done. mem= 2.901 Gb
concept= 18 (418 words) ... done. mem= 2.914 Gb
concept= 19 (474 words) ... done. mem= 2.925 Gb
concept= 20 (444 words) ... done. mem= 2.935 Gb
co

## temporal saving

In [29]:
if False:
    import pickle
    M_term_doc_fname = '../data/M_term_doc.pkl'
    M_cooccurrence_fname = '../data/M_cooccurrence.pkl'

    with open(M_term_doc_fname, 'wb') as f:
        pickle.dump(dict(M_term_doc), f)

    with open(M_cooccurrence_fname, 'wb') as f:
        pickle.dump(dict(M_cooccurrence), f)

\begin{align}
 S(X) = \frac{\frac{k^{(1/m)}}{k^{(1/n)}} -\frac{\|X\|_m}{\|X\|_n} } {\frac{k^{(1/m)}}{k^{(1/n)}}-1}
\end{align}

- k: co-occurrance vector dimension: number of words in a concept
- m: 1 why? 빠를거 같아서
- n: 2 빠를거 같아서
- X: co-occurrance vector (in all documents)

In [19]:
norm = lambda x, p: 0 if (not x or p == 0) else pow(sum(v ** p for v in x.values()), 1/p)
sparsity = lambda x, m, n, km_over_kn: (km_over_kn - (norm(x,m)/norm(x,n))) / (km_over_kn - 1)


In [32]:
word_to_sparsity = {}

m = 1
n = 2

i_words = 0
n_words = sum((len(words) for words in concept_to_words.values()))

for concept, words in concept_to_words.items():
    k = len(words)
    km_over_kn = pow(k, 1/m) / pow(k, 1/n)
    
    for word in words:
        i_words += 1
        if i_words % 100 == 0:
            args = (i_words, n_words, get_process_memory())
            sys.stdout.write('\r computing sparsity ... %d words in %d. mem= %.3f Gb' % args)
        
        cooccurrence_vector = M_cooccurrence.get(word, {})
        
        if not cooccurrence_vector:
            continue
            
        if len(cooccurrence_vector) == 1:
            word_to_sparsity[word] = 1
            continue
            
        word_to_sparsity[word] = sparsity(cooccurrence_vector, m, n, km_over_kn)
print('\ndone')

 computing sparsity ... 40000 words in 40069. mem= 3.615 Gb
done


In [33]:
if False:
    import pickle
    sparsity_pickle = '../data/word_to_sparsity.pkl'

    with open(sparsity_pickle, 'wb') as f:
        pickle.dump(word_to_sparsity, f)

In [40]:
get_sparsity = lambda w:word_to_sparsity[w]
get_tf = lambda w:sum(M_term_doc[w].values())
get_df = lambda w:len(M_term_doc[w])

for concept, words in concept_to_words.items():
    topk_words = sorted(words, key=lambda x:word_to_sparsity[x])[:10]
    print('concept= %d: %s\n' % (concept, '\n\t\t'.join(['(%s, s=%.3f, tf=%d, df=%d)' % (w, get_sparsity(w), get_tf(w), get_df(w)) for w in topk_words])))

concept= 0: (unleashed, s=0.482, tf=490, df=478)
		(blew, s=0.488, tf=1104, df=1042)
		(barrage, s=0.489, tf=434, df=426)
		(fought, s=0.491, tf=3344, df=3101)
		(swept, s=0.493, tf=1746, df=1682)
		(ripped, s=0.493, tf=452, df=434)
		(blasted, s=0.494, tf=860, df=838)
		(steered, s=0.494, tf=523, df=516)
		(blown, s=0.495, tf=581, df=555)
		(battling, s=0.496, tf=2307, df=2235)

concept= 1: (nobel, s=0.699, tf=1412, df=925)
		(thesis, s=0.708, tf=162, df=141)
		(physics, s=0.708, tf=324, df=241)
		(director, s=0.717, tf=30777, df=22891)
		(distinguished, s=0.717, tf=222, df=212)
		(mit, s=0.718, tf=200, df=134)
		(chemistry, s=0.722, tf=508, df=414)
		(phd, s=0.724, tf=141, df=125)
		(scholar, s=0.727, tf=378, df=354)
		(academia, s=0.728, tf=96, df=90)

concept= 2: (party, s=0.697, tf=31344, df=15323)
		(renzi, s=0.702, tf=406, df=111)
		(centre-right, s=0.709, tf=400, df=331)
		(centre-left, s=0.711, tf=339, df=244)
		(coalition, s=0.712, tf=7969, df=4749)
		(left-wing, s=0.716, tf=